In [6]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import logging
from itertools import product


class DataFetcher:
    def __init__(self, tickers, start='2023-01-01', end='2024-01-01'):
        self.tickers = tickers
        self.start = start
        self.end = end
    
    def fetch_data(self, ticker):
        logging.info(f"Fetching data for {ticker}")
        df = yf.download(ticker, start=self.start, end=self.end,multi_level_index=False,progress=False)
        if df.empty:
            logging.warning(f"No data found for {ticker}")
        return df

# Nifty 50 tickers
nifty_50_tickers = [
    "ADANIENT.NS", "ADANIPORTS.NS", "APOLLOHOSP.NS", "ASIANPAINT.NS", "AXISBANK.NS",
    "BAJAJ-AUTO.NS", "BAJFINANCE.NS", "BAJAJFINSV.NS", "BEL.NS", "BPCL.NS",
    "BHARTIARTL.NS", "BRITANNIA.NS", "CIPLA.NS", "COALINDIA.NS", "DRREDDY.NS",
    "EICHERMOT.NS", "GRASIM.NS", "HCLTECH.NS", "HDFCBANK.NS", "HDFCLIFE.NS",
    "HEROMOTOCO.NS", "HINDALCO.NS", "HINDUNILVR.NS", "ICICIBANK.NS", "INDUSINDBK.NS",
    "INFY.NS", "ITC.NS", "JSWSTEEL.NS", "KOTAKBANK.NS", "LT.NS",
    "M&M.NS", "MARUTI.NS", "NESTLEIND.NS", "NTPC.NS", "ONGC.NS",
    "POWERGRID.NS", "RELIANCE.NS", "SBILIFE.NS", "SHRIRAMFIN.NS", "SBIN.NS",
    "SUNPHARMA.NS", "TCS.NS", "TATACONSUM.NS", "TATAMOTORS.NS", "TATASTEEL.NS",
    "TECHM.NS", "TITAN.NS", "TRENT.NS", "ULTRACEMCO.NS", "WIPRO.NS"
]

class SMAMomentumStrategy:
    def __init__(self, df, fee=0.00075):
        self.data = df.copy().reset_index(drop=True)
        self.fee = fee
        self.trades = 0
    
    def compute_indicators(self, ema_window, momentum_window):
        close_price = self.data['Close']
        self.data['sma'] = close_price.ewm(span=ema_window, adjust=False).mean()
        self.data['momentum'] = close_price.pct_change(momentum_window)
    
    def run(self, sma_window, momentum_window, momentum_thresh):
        self.compute_indicators(sma_window, momentum_window)
        
        signals = np.zeros(len(self.data))
        cash, stock, position = 1000.0, 0.0, 0
        
        for i in range(1, len(self.data)):
            price, sma_val, mom_val = self.data.loc[i, 'Close'], self.data.loc[i-1, 'sma'], self.data.loc[i-1, 'momentum']
            if np.isnan(sma_val) or np.isnan(mom_val):
                continue
            
            if position == 0 and price > sma_val and mom_val > momentum_thresh:
                stock, cash, position = cash / (price * (1 + self.fee)), 0.0, 1
                signals[i] = 1
                self.trades += 1
            elif position == 1 and price < sma_val and mom_val < -momentum_thresh:
                cash, stock, position = stock * price * (1 - self.fee), 0.0, 0
                signals[i] = -1
                self.trades += 1
        
        if position == 1:
            cash = stock * self.data['Close'].iloc[-1] * (1 - self.fee)
        
        return signals, cash, self.trades


class Backtester:
    def __init__(self, tickers, start='2023-01-01', end='2024-01-01'):
        self.tickers = tickers
        self.start = start
        self.end = end
        self.data_fetcher = DataFetcher(tickers, start, end)
    
    def optimize_strategy(self, df):
        ema_windows = [10, 20, 30, 50, 90, 150]
        momentum_windows = [10, 20, 30, 50, 90, 150]
        momentum_thresholds = [0.01, 0.03,0.05, 0.08]
        
        best_value, best_params = -np.inf, None
        train_size = int(0.7 * len(df))
        train_df = df.iloc[:train_size]
        
        for sma, mom, thresh in product(ema_windows, momentum_windows, momentum_thresholds):
            strategy = SMAMomentumStrategy(train_df)
            _, final_value, _ = strategy.run(sma, mom, thresh)
            if final_value > best_value:
                best_value, best_params = final_value, (sma, mom, thresh)
        
        return best_params
    
    def compute_metrics(self, df, final_value, trades):
        initial_cash = 1000.0
        buy_and_hold_return = df['Close'].iloc[-1] / df['Close'].iloc[0] - 1
        strategy_return = final_value / initial_cash - 1
        returns = df['Close'].pct_change().dropna()
        sharpe_ratio = (returns.mean() / returns.std()) * np.sqrt(252) if returns.std() != 0 else 0
        return buy_and_hold_return, strategy_return, sharpe_ratio, trades
    
    def run(self):
            results_data = []
            for ticker in self.tickers:
                df = self.data_fetcher.fetch_data(ticker)
                if df.empty:
                    continue
                
                best_params = self.optimize_strategy(df)
                test_df = df.iloc[int(0.7 * len(df)):]  # Test set
                
                strategy = SMAMomentumStrategy(test_df)
                signals, final_value, trades = strategy.run(*best_params)
                
                buy_and_hold, strat_return, sharpe, trade_count = self.compute_metrics(test_df, final_value, trades)
                
                results_data.append({
                    'Ticker': ticker,
                    'Final Value': final_value,
                    'Buy & Hold Return': buy_and_hold,
                    'Strategy Return': strat_return,
                    'Sharpe Ratio': sharpe,
                    'Trades Taken': trade_count,
                    'Best Params': best_params
                })
            
            results_df = pd.DataFrame(results_data)

            # Function to color rows
            def highlight_rows(row):
                if row['Strategy Return'] > row['Buy & Hold Return']:
                    return ['background-color: lightgreen'] * len(row)
                else:
                    return ['background-color: lightcoral'] * len(row)

            # Apply styling
            styled_df = results_df.style.apply(highlight_rows, axis=1)
            
            # Display the styled DataFrame
            display(styled_df)
            
            return results_df
    

# Run backtest
backtester = Backtester(nifty_50_tickers)
results_df = backtester.run()



,Ticker,Final Value,Buy & Hold Return,Strategy Return,Sharpe Ratio,Trades Taken,Best Params
0,ADANIENT.NS,986.383199,0.089485,-0.013617,0.858608,1,"(20, 30, 0.08)"
1,ADANIPORTS.NS,1220.979616,0.160802,0.220980,1.484636,1,"(10, 20, 0.03)"
2,APOLLOHOSP.NS,1061.256263,0.119340,0.061256,1.825193,1,"(150, 50, 0.01)"
3,ASIANPAINT.NS,1101.017710,0.045695,0.101018,1.024492,1,"(30, 10, 0.03)"
4,AXISBANK.NS,1077.271005,0.101859,0.077271,1.694263,1,"(50, 10, 0.03)"
5,BAJAJ-AUTO.NS,1348.525580,0.414208,0.348526,4.965241,1,"(150, 10, 0.01)"
6,BAJFINANCE.NS,966.133823,-0.008645,-0.033866,-0.025593,4,"(30, 10, 0.01)"
7,BAJAJFINSV.NS,985.417775,0.087824,-0.014582,1.508873,6,"(10, 10, 0.01)"
8,BEL.NS,1278.581198,0.272979,0.278581,2.948119,1,"(10, 20, 0.05)"
9,BPCL.NS,1306.984250,0.297431,0.306984,3.509144,1,"(10, 30, 0.01)"
